In [2]:
import os
import numpy as np

from emtools import BaseClasses

In [3]:
nc_dir = r"C:\Users\nnolde\Documents\Python27\Projects\EM\MARE2DEM\Rotated"
ex_nc = None
nc_list = []
nc_names = []
with open(os.path.join(nc_dir, '01Rx001a_Rotated.cdl')) as r:

    for fd, sub, files in os.walk(nc_dir):
        for f in files:
            if f.endswith('.nc'):
                ex_nc = BaseClasses.nc(os.path.join(nc_dir, f), 'r')
                nc_list.append(ex_nc)
                nc_names.append(f)

# Start the .emdata definition

In [4]:
out_dir = r'C:\Users\nnolde\Documents\Python27\Projects\EM\MARE2DEM\emdata\{}'.format(nc_names[0])

In [84]:
ex_nc = nc_list[0]
print ex_nc.printAttrs()

version: 1.1.0
logid: unknown
surveyid: STOS
jobid: unknown
recid: 01Rx001a
lineid: 01Tx002a
domain: frequency
coilid: unknown
x_r: 1.63349e+06
y_r: 5.61939e+06
z_r: 109.4
psi: 0.0
phi: 44.2604
theta: 0.0
psi_uncertainty: 0.0
phi_uncertainty: 3.0
theta_uncertainty: 0.0
receiver_heading: -1000.0
sigma_f: 0.0
dataset_date: Jan Feb 2011
dataset_author: Oystein Hallanger-Martin Hansen
dlength: 270.52
geodetic_datum: unknown
projection_type: transverse_mercator_north_oriented
projection_zone: unknown
long_of_central_meridian: 173deg 00min 00.000sec E
grid_origin: 000deg 00min 00.000sec N,   173deg 00min 00.000sec E
grid_coord_at_origin: 1600000.00E,  10000000.00N
scale_factor: unknown
scale_factor_long_lat: 0.99960000000000004
planned_vessel_speed: 2.00 kts
format_type: complex_demod
t0_emgs: 17-01-2011 08:35:43
t0_unix: 1295253343.0
nf: 10
freqs: [ 0.11160714  0.33482143  0.55803573  0.78125     1.00446427  1.22767854
  1.45089281  1.67410719  1.89732146  2.12053561]
dx1: 0.0
dx2: 0.0
dk1:

## Header

In [160]:
rx_x = ex_nc.x_r
rx_y = ex_nc.y_r
src_hd = float(ex_nc.approx_source_heading)

str_out = """Format:  EMData_2.2
!real
UTM of x,y origin (UTM zone, N, E, 2D strike): 0 N{0:>13.10g}{1:>13.10g}{2:>7.5g}
Phase Convention: lag
Reciprocity Used:""".format(rx_x, rx_y, src_hd - 90)

## Frequencies

In [161]:
freqs_amount = len(ex_nc.freqs)

str_out += """
# CSEM Frequencies:    {0}
{1}""".format(freqs_amount, "\n".join([str(x) for x in ex_nc.freqs]))

## Transmitters

In [162]:
src_pos = ex_nc.variables['srcpos']
tx_id = ex_nc.lineid
print ex_nc.variables['srcpos'].srcpos_direction
print src_pos[:1]

x y z wd hd _
[[  1.62624438e+06   5.61192800e+06   1.06417713e+01   1.15933388e+02
    4.22444916e+01   9.96920997e+36]]


In [163]:
str_out += """
# Transmitters:   1
!          X            Y            Z Azimuth     Dip  Length    Type  Name 
{0:>12.10g}{1:>13.10g}{2:13.10g}{3:>8.6g}{4:>8.6g}{5:>8.6g}{6:>9}{7:>5}""".format(src_pos[0][0],
                                                                                 src_pos[0][1],
                                                                                 src_pos[0][2],
                                                                                 src_hd,
                                                                                 0.00,
                                                                                 ex_nc.dlength,
                                                                                 "edipole",
                                                                                 ex_nc.lineid[-3:])

## Receivers

In [164]:
str_out += """
# CSEM Receivers:      {}
!          X            Y            Z   Theta   Alpha    Beta  Length  Name""".format(len(nc_list))

for rx in nc_list:
    str_out += """
{0:>12.10g}{1:>13.10g}{2:>13.10g}{3:>8}{4:>8}{5:>8}{6:>8}{7:>6}""".format(float(rx.x_r),
                                                                                   float(rx.y_r),
                                                                                   float(rx.z_r),
                                                                                   "0.00",
                                                                                   "0.00",
                                                                                   "0.00",
                                                                                   "0.00",
                                                                                   rx.recid[-3:])

## Data

# Write .emdata file

In [165]:
with open(out_dir, 'w') as f:
    f.write(str_out)